In [ ]:
# !pip install louvain
# !pip install --upgrade --no-cache psutil gputil

In [1]:
import os
import subprocess
import sys
sys.path.append('..')
from tools.formating.formating import *
from tools.utils.datasplit import *
from tools.qc.scanpy_qc import *
import numpy as np
import scipy

R[write to console]: Loading required package: SingleCellExperiment

R[write to console]: Loading required package: SummarizedExperiment

R[write to console]: Loading required package: MatrixGenerics

R[write to console]: Loading required package: matrixStats

R[write to console]: 
Attaching package: ‘MatrixGenerics’


R[write to console]: The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowC


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

R[write to console]: Attaching SeuratObject

R[write to console]: Seurat v4 was just loaded with SeuratObject v5; disabling v5 assays and
validation routines, and ensuring assays work in strict v3/v4
compatibility mode

R[write to console]: 
Attaching package: ‘Seurat’


R[write to console]: The following object is masked from ‘package:SummarizedExperiment’:

    Assays


R[write to console]: Registered S3 method overwritten by 'SeuratDisk':
  method            from  
  as.sparse.H5Group Seurat

R[write to console]: Signac built for for SeuratObject v4 was just loaded with SeuratObject
v5; disabling v5 assays and validation routines, and ensuring assays
work in strict v3/v4 compatibility mode



scanpy==1.9.5 anndata==0.9.2 umap==0.5.4 numpy==1.22.4 scipy==1.9.1 pandas==1.3.5 scikit-learn==1.3.1 statsmodels==0.14.0 igraph==0.10.8 louvain==0.8.1 pynndescent==0.5.10


In [21]:
# input = "/ps/ai-ready/data/Clustering/Tabula_Sapiens/TS_germ line.h5ad"
# input = "/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5ad"
# input = "/ps/ai-ready/data/error/hca_heart_neuronal_raw.h5ad"
# input = "/ps/ai-ready/data/Clustering/Tabula_Sapiens/TS_Skin.h5ad"
input = "/ps/ai-ready/data/error/droplet_Bladder_seurat_tiss.h5seurat"
# input = "/ps/ai-ready/data/error/tung.rds"
adata = load_anndata(input)
adata

/ps/ai-ready/data/error/droplet_Bladder_seurat_tiss.h5seurat
convert_seurat_sce_to_anndata
RNA
[1] "Inside ConvertSeuratSCEtoAnndata R function"
[1] "Inside LoadSeurat"


R[write to console]: Validating h5Seurat file

R[write to console]: Initializing RNA with data

R[write to console]: Adding counts for RNA

R[write to console]: Adding miscellaneous information for RNA

R[write to console]: Adding command information

R[write to console]: Adding cell-level metadata

R[write to console]: Adding miscellaneous information

R[write to console]: Adding tool-specific results



[1] "srat object is loaded"
[1] "AnnData object is saved successfully."


AnnData object with n_obs × n_vars = 2500 × 23341
    obs: 'orig.ident', 'n_counts', 'n_genes', 'channel', 'tissue', 'subtissue', 'mouse.sex', 'mouse.id', 'percent.ercc', 'percent.ribo', 'free_annotation', 'cell_ontology_class', 'res.0.4', 'previous_free_annotation', 'previous_cell_ontology_class', 'cluster.ids', 'cell_ontology_id'
    var: 'name'

In [22]:
adata.obs

,orig.ident,n_counts,n_genes,channel,tissue,subtissue,mouse.sex,mouse.id,percent.ercc,percent.ribo,free_annotation,cell_ontology_class,res.0.4,previous_free_annotation,previous_cell_ontology_class,cluster.ids,cell_ontology_id
10X_P4_3_AAAGTAGAGATGCCAG,10X,10291.0,2853,10X_P4_3,Bladder,,M,3-M-8,0.0,0.219609,Bladder mesenchymal cell,bladder cell,0,NA,NA,0,CL:1001319
10X_P4_3_AACCGCGTCCAACCAA,10X,17520.0,4194,10X_P4_3,Bladder,,M,3-M-8,0.0,0.176712,Bladder mesenchymal cell,bladder cell,2,NA,NA,2,CL:1001319
10X_P4_3_AACTCCCGTCGGGTCT,10X,12944.0,3347,10X_P4_3,Bladder,,M,3-M-8,0.0,0.192444,Bladder mesenchymal cell,bladder cell,5,NA,NA,5,CL:1001319
10X_P4_3_AACTCTTAGTTGCAGG,10X,19594.0,3851,10X_P4_3,Bladder,,M,3-M-8,0.0,0.356385,Luminal bladder epithelial cell,bladder urothelial cell,3,NA,NA,3,CL:1001428
10X_P4_3_AACTCTTTCATAACCG,10X,14594.0,3809,10X_P4_3,Bladder,,M,3-M-8,0.0,0.157119,Bladder mesenchymal cell,bladder cell,2,NA,NA,2,CL:1001319
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10X_P7_7_TTGTAGGTCATGTAGC,10X,22993.0,4296,10X_P7_7,Bladder,,F,3-F-56,0.0,0.286783,Basal bladder epithelial cell,bladder urothelial cell,1,NA,NA,1,CL:1001428
10X_P7_7_TTTACTGGTGTTGAGG,10X,8753.0,3005,10X_P7_7,Bladder,,F,3-F-56,0.0,0.120873,Bladder mesenchymal cell,bladder cell,5,NA,NA,5,CL:1001319
10X_P7_7_TTTCCTCCATACCATG,10X,14665.0,3778,10X_P7_7,Bladder,,F,3-F-56,0.0,0.118173,Bladder mesenchymal cell,bladder cell,5,NA,NA,5,CL:1001319
10X_P7_7_TTTGTCAGTTACCGAT,10X,30363.0,4786,10X_P7_7,Bladder,,F,3-F-56,0.0,0.307018,Basal bladder epithelial cell,bladder urothelial cell,1,NA,NA,1,CL:1001428


In [ ]:
adata.obs.index.get_loc('TCGACGGGTACGAGCA-1-HCAHeart8287128')

In [ ]:
type(adata.X)

In [ ]:
adata.X.toarray()

In [ ]:
adata.layers['raw_counts'].toarray()

In [ ]:
adata.raw.X.toarray()

In [ ]:
"raw_counts" in adata.layers.keys()

In [ ]:
isinstance(adata.X, csr_matrix)

In [ ]:
np.set_printoptions(threshold=np.inf)
adata.X.toarray()[5,:]

In [ ]:
type(adata.X.toarray())

In [ ]:
isinstance(adata.X.toarray(), np.ndarray)

In [ ]:
np.max(adata.X)

In [ ]:
np.min(adata.X)

In [ ]:
from scipy.sparse import csr_matrix, find
find(adata.X)

In [ ]:
from scipy import stats
stats.describe(adata.X.toarray())

In [ ]:
np.median(adata.X.toarray())

In [14]:
min_genes=200
min_cells=3
target_sum=1e4
n_neighbors=10
n_pcs=40
max_genes=0

In [24]:
sc.pp.filter_cells(adata, min_genes=min_genes)
if max_genes is not None and max_genes != 0:
    sc.pp.filter_cells(adata, max_genes=max_genes)
sc.pp.filter_genes(adata, min_cells=min_cells)

filtered out 7956 genes that are detected in less than 3 cells


In [26]:
adata.var['mt']=adata.var_names.str.startswith('MT-')
adata.var["ribo"] = adata.var_names.str.startswith(("RPS", "RPL"))
adata.var["hb"] = adata.var_names.str.contains(("^HB[^(P)]"))

In [25]:
adata.var

,name,n_cells
Xkr4,Xkr4,4
Sox17,Sox17,38
Mrpl15,Mrpl15,766
Lypla1,Lypla1,948
Tcea1,Tcea1,1452
...,...,...
Kdm5d,Kdm5d,148
Eif2s3y,Eif2s3y,784
Uty,Uty,136
Ddx3y,Ddx3y,1316


In [20]:
adata.obs

,orig.ident,n_counts,n_genes,channel,tissue,subtissue,mouse.sex,mouse.id,percent.ercc,percent.ribo,...,total_counts_ribo,log1p_total_counts_ribo,pct_counts_ribo,total_counts_hb,log1p_total_counts_hb,pct_counts_hb,outlier,mt_outlier,doublet_scores,predicted_doublets


In [27]:
sc.pp.calculate_qc_metrics(adata, qc_vars=["mt", "ribo", "hb"], inplace=True, percent_top=[20], log1p=True)

In [4]:
adata.var_names_make_unique() 

# Filtering low quality reads
sc.pp.filter_cells(adata, min_genes=min_genes)
sc.pp.filter_genes(adata, min_cells=min_cells)
# mitochondrial genes
adata.var['mt']=adata.var_names.str.startswith('MT-')
# ribosomal genes
adata.var["ribo"] = adata.var_names.str.startswith(("RPS", "RPL"))
# hemoglobin genes.
adata.var["hb"] = adata.var_names.str.contains(("^HB[^(P)]"))
sc.pp.calculate_qc_metrics(adata, qc_vars=["mt", "ribo", "hb"], inplace=True, percent_top=[20], log1p=True)

adata.obs["outlier"] = (
    is_outlier(adata, "log1p_total_counts", 5)
    | is_outlier(adata, "log1p_n_genes_by_counts", 5)
    | is_outlier(adata, "pct_counts_in_top_20_genes", 5)
)
adata.obs.outlier.value_counts()

adata.obs["mt_outlier"] = is_outlier(adata, "pct_counts_mt", 3) | (
    adata.obs["pct_counts_mt"] > 8
)
adata.obs.mt_outlier.value_counts()

print(f"Total number of cells: {adata.n_obs}")
adata = adata[(~adata.obs.outlier) & (~adata.obs.mt_outlier)].copy()

print(f"Number of cells after filtering of low quality cells: {adata.n_obs}")
adata.X

filtered out 7956 genes that are detected in less than 3 cells
Total number of cells: 2500
Number of cells after filtering of low quality cells: 2441


<2441x15385 sparse matrix of type '<class 'numpy.float32'>'
	with 7513087 stored elements in Compressed Sparse Column format>

In [6]:
# adata=adata[adata.obs.n_genes_by_counts < 2500, :]
# adata=adata[adata.obs.pct_counts_mt < 5, :]
import scrublet as scr
scrub = scr.Scrublet(adata.X, expected_doublet_rate = 0.076)
adata.obs['doublet_scores'], adata.obs['predicted_doublets'] = scrub.scrub_doublets(min_counts=2, min_cells=3, 
                                                        min_gene_variability_pctl=85, n_prin_comps=30)
adata.obs['predicted_doublets'].value_counts()
# adata=adata[adata.obs.predicted_doublets=="False", :]

sc.pp.normalize_total(adata, target_sum=target_sum)

sc.pp.log1p(adata)

Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.41
Detected doublet rate = 0.7%
Estimated detectable doublet fraction = 45.9%
Overall doublet rate:
	Expected   = 7.6%
	Estimated  = 1.5%
Elapsed time: 4.7 seconds
normalizing counts per cell
    finished (0:00:00)


In [ ]:
adata.X

In [ ]:
sc.pp.highly_variable_genes(adata)

In [7]:
import scrublet as scr
scrub = scr.Scrublet(adata.X, expected_doublet_rate = 0.076)
adata.obs['doublet_scores'], adata.obs['predicted_doublets'] = scrub.scrub_doublets(min_counts=2, min_cells=3, 
                                                        min_gene_variability_pctl=85, n_prin_comps=30)
adata.obs['predicted_doublets'].value_counts()

Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.44
Detected doublet rate = 0.5%
Estimated detectable doublet fraction = 52.9%
Overall doublet rate:
	Expected   = 7.6%
	Estimated  = 0.9%
Elapsed time: 2.7 seconds


False    2430
True       11
Name: predicted_doublets, dtype: int64

In [ ]:
adata=adata[adata.obs.predicted_doublets==False, :]
adata

In [ ]:
adata = run_scanpy_qc(adata)
adata

In [ ]:
adata.X

In [ ]:
isinstance(adata.X, np.ndarray)

In [ ]:
adata = run_dimension_reduction(adata, layer="log10k")
adata

In [ ]:
adata.layers["raw_counts"].toarray()

In [ ]:
adata.X.toarray()

In [ ]:
adata.layers["log10k"] = adata.X

In [ ]:
del adata.layers["X"]
adata

In [ ]:
adata.uns

In [ ]:
adata.obs

In [ ]:
adata.obsm['log10k_umap'].shape

In [ ]:
adata.raw.X.toarray()

In [ ]:
adata.var.index

In [ ]:
type(adata.obs['total_counts'].tolist())

In [ ]:
adata.obs['n_counts']

In [ ]:
'pct_counts_mt' in adata.obs.columns

In [ ]:
adata.obs_vector('pct_counts_mt')

In [ ]:
train, test = sc_train_test_split(adata, train_fraction=0.8)

In [ ]:
train, val, test = sc_train_val_test_split(adata, train_fraction=0.8, validation_fraction=0.1, test_fraction=0.1)

In [ ]:
train

In [ ]:
val

In [ ]:
test

In [ ]:
test.X.toarray()

In [ ]:
test.layers["raw_counts"].toarray()

In [ ]:
test.write_h5ad(filename="/ps/ai-ready/data/Clustering/Tabula_Sapiens/TS_Skin_test.h5ad", compression='gzip')

In [ ]:
adata = run_dimension_reduction(adata)

In [ ]:
# from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score as ARI
from sklearn.metrics import normalized_mutual_info_score as NMI
from sklearn.metrics import silhouette_score
# from sklearn.mixture import GaussianMixture as GMM

def clustering_scores(labels, labels_pred, embedding):
    asw_score = silhouette_score(embedding, labels)
    nmi_score = NMI(labels, labels_pred)
    ari_score = ARI(labels, labels_pred)
    print(
        "Clustering Scores:\nSilhouette: %.4f\nNMI: %.4f\nARI: %.4f"
        % (asw_score, nmi_score, ari_score)
    )
    return asw_score, nmi_score, ari_score

In [ ]:
asw_score, nmi_score, ari_score = clustering_scores(adata.obs["cell_ontology_class"], adata.obs["leiden"], adata.obsp['connectivities'])

In [ ]:
adata.obsp['connectivities'].toarray()

In [ ]:
test.layers["log10k"].shape

In [ ]:
from scipy.sparse import csr_matrix
test.X = csr_matrix(test.X)
test.X

In [ ]:
test.layers["raw_counts"].toarray().shape

In [ ]:
adata.var

In [ ]:
list(adata.layers.keys())

In [ ]:
adata.n_vars

In [ ]:
adata.n_obs

In [ ]:
adata.obs_names.to_list()

In [ ]:
adata.var_names.to_list()

In [ ]:
list(adata.obsm.keys())

In [ ]:
adata.X

In [ ]:
type(adata.obs_keys())

In [ ]:
adata.var_keys()

In [ ]:
random_state=0
fraction=0.8
np.random.seed(random_state)
old_n_obs = adata.n_obs
old_n_obs
new_n_obs = int(fraction * old_n_obs)
obs_indices = np.random.choice(old_n_obs, size=new_n_obs, replace=False)
obs_indices

In [ ]:
test=adata[obs_indices].copy()
test

In [ ]:
test.var

In [ ]:
original_indices=np.arange(old_n_obs)
original_indices

In [ ]:
diff=np.setdiff1d(original_indices, obs_indices)
diff

In [ ]:
len(diff)

In [ ]:
np.random.shuffle(diff)
diff

In [ ]:
train=adata[diff].copy()
train

In [ ]:
train.obs